In [ ]:
import gurobipy as gp
from gurobipy import GRB
import re
import itertools

# -----------------------------
# INPUT DATA (as provided)
# -----------------------------
courses = {
    'Owen Tyler Dodd': [
        {'CRN': 30522, 'days': ['T', 'R'], 'demand': 41}
    ],
    'Rui Zhu': [
        {'CRN': 30524, 'days': ['M', 'W'], 'demand': 44}
    ],
    'Talayeh Razzaghi': [
        {'CRN': 43516, 'days': ['T', 'R'], 'demand': 17}
    ],
    'Theodore B Trafalis': [
        {'CRN': 41627, 'days': ['T', 'R'], 'demand': 9}
    ],
    'Shivakumar Raman': [
        {'CRN': 30529, 'days': ['T', 'R'], 'demand': 44},
        {'CRN': 30530, 'days': 'Lab', 'demand': 16},
        {'CRN': 30532, 'days': 'Lab', 'demand': 16},
        {'CRN': 30533, 'days': 'Lab', 'demand': 8},
        {'CRN': 34447, 'days': 'Lab', 'demand': 5},
        {'CRN': 42741, 'days': 'Lab', 'demand': 4}
    ],
    'Charles D Nicholson': [
        {'CRN': 30535, 'days': ['T', 'R'], 'demand': 46},
        {'CRN': 34608, 'days': ['T', 'R'], 'demand': 31}
    ],
    'Cesar Alexander Ruiz Torres': [
        {'CRN': 30536, 'days': ['T', 'R'], 'demand': 39}
    ],
    'Kash A Barker': [
        {'CRN': 32173, 'days': ['M', 'W', 'F'], 'demand': 40},
        {'CRN': 30539, 'days': ['M', 'W'], 'demand': 65}
    ],
    'Andres D Gonzalez': [
        {'CRN': 30541, 'days': ['T', 'R'], 'demand': 70},
        {'CRN': 30561, 'days': ['T', 'R'], 'demand': 20}
    ],
    'Ziho Kang': [
        {'CRN': 30544, 'days': ['M', 'W'], 'demand': 39},
        {'CRN': 30548, 'days': 'Lab', 'demand': 15},
        {'CRN': 30550, 'days': 'Lab', 'demand': 13},
        {'CRN': 42308, 'days': 'Lab', 'demand': 19},
        {'CRN': 30556, 'days': ['M', 'W', 'F'], 'demand': 45},
        {'CRN': 30562, 'days': ['M', 'W'], 'demand': 10}
    ]
}

rooms = {
    '324': {'capacity': 10, 'building': 'BL'},
    '117': {'capacity': 55, 'building': 'CEC'},
    '121': {'capacity': 25, 'building': 'CEC'},
    '205': {'capacity': 20, 'building': 'CEC'},
    '438': {'capacity': 25, 'building': 'CEC'},
    'S0014': {'capacity': 20, 'building': 'CEC'},
    '300': {'capacity': 100, 'building': 'FH'},
    '304': {'capacity': 100, 'building': 'FH'},
    '336': {'capacity': 100, 'building': 'FH'},
    '123': {'capacity': 120, 'building': 'GLCH'},
    '127': {'capacity': 130, 'building': 'GLG'},
    '108': {'capacity': 80, 'building': 'PHSC'},
    'M0204': {'capacity': 70, 'building': 'SEC'},
    'N0202': {'capacity': 80, 'building': 'SEC'},
    'P0201': {'capacity': 50, 'building': 'SEC'}
}

def get_pattern(days):
    if days == 'Lab':
        return 'Lab'
    dset = set(days)
    if dset == {'T','R'}:
        return 'TR'
    elif dset == {'M','W'}:
        return 'MW'
    elif dset == {'M','W','F'}:
        return 'MWF'
    else:
        return 'Lab'

AllDays = ['M','T','W','R','F']

time_slots = {
    'M': {0: '9:30 AM - 10:20 AM',1:'10:30 AM - 11:20 AM',2:'11:30 AM - 12:20 PM',3:'1:30 PM - 2:45 PM',4:'3:00 PM - 4:15 PM'},
    'T': {0:'7:30 AM - 8:45 AM',1:'9:00 AM - 10:15 AM',2:'10:30 AM - 11:45 AM',3:'12:00 PM - 1:15 PM',4:'1:30 PM - 2:45 PM',5:'3:00 PM - 4:15 PM',6:'4:30 PM - 5:45 PM'},
    'W': {0:'9:30 AM - 10:20 AM',1:'10:30 AM - 11:20 AM',2:'11:30 AM - 12:20 PM',3:'1:30 PM - 2:45 PM',4:'3:00 PM - 4:15 PM'},
    'R': {0:'7:30 AM - 8:45 AM',1:'9:00 AM - 10:15 AM',2:'10:30 AM - 11:45 AM',3:'12:00 PM - 1:15 PM',4:'1:30 PM - 2:45 PM',5:'3:00 PM - 4:15 PM',6:'4:30 PM - 5:45 PM'},
    'F': {0:'9:30 AM - 10:20 AM',1:'10:30 AM - 11:20 AM',2:'11:30 AM - 12:20 PM'}
}

# Add distinct lab slot for each day
time_slots['M'][5] = '4:30 PM - 5:30 PM (Lab)'
time_slots['T'][7] = '4:30 PM - 5:30 PM (Lab)'
time_slots['W'][5] = '4:30 PM - 5:30 PM (Lab)'
time_slots['R'][7] = '4:30 PM - 5:30 PM (Lab)'
time_slots['F'][3] = '4:30 PM - 5:30 PM (Lab)'

lab_slot = {'M':5,'T':7,'W':5,'R':7,'F':3}

Classes = []
for instr,clist in courses.items():
    clean_instr = instr.strip()
    for c in clist:
        CRN = c['CRN']
        days = c['days']
        dem = c['demand']
        pattern = get_pattern(days)
        # If pattern='Lab', days=AllDays in tuple
        DD = tuple(days) if pattern!='Lab' else tuple(AllDays)
        Classes.append((CRN, clean_instr, pattern, DD, dem))

ClassInfo = {c[0]: c for c in Classes}
FeasibleRooms = {CRN: [r for r in rooms if rooms[r]['capacity']>=ClassInfo[CRN][4]] for CRN in ClassInfo}

# FeasibleDayTime calculation:
FeasibleDayTime = {}
for CRN,(C,I,P,DD,dem) in ClassInfo.items():
    feasible = []
    if P=='Lab':
        # Only lab slots each day
        for d in AllDays:
            feasible.append((d, lab_slot[d], P))
    else:
        # For pattern classes, exclude the lab slot
        for d in DD:
            for t_idx in time_slots[d].keys():
                if t_idx != lab_slot[d]:
                    feasible.append((d,t_idx,P))
    FeasibleDayTime[CRN] = feasible

OfficeBuilding = "CEC"
W_intra = 7
W_inter = 12

# Parse times to minutes
def to_minutes(h,mm,ampm):
    hh = int(h)
    m = int(mm)
    if ampm=='PM' and hh<12:
        hh+=12
    return hh*60+m

def start_end_times(d,t):
    slot_str = time_slots[d][t]
    # format: H:MM AM - H:MM AM/PM (Careful with PM)
    m1 = re.match(r'(\d+):(\d+) (\wM) - (\d+):(\d+) (\wM)', slot_str)
    h1,mn1,ap1 = m1.group(1),m1.group(2),m1.group(3)
    h2,mn2,ap2 = m1.group(4),m1.group(5),m1.group(6)
    startM = to_minutes(h1,mn1,ap1)
    endM   = to_minutes(h2,mn2,ap2)
    return startM,endM

def building_of_room(r):
    return rooms[r]['building']

def cost_between_buildings(b1, b2, OfficeBuilding="CEC", W_intra=7, W_inter=12):
    if b1 == b2:
        return W_intra
    if b1==OfficeBuilding or b2==OfficeBuilding:
        if b1!=b2:
            return W_inter
        else:
            return W_intra
    return W_inter

def is_consecutive(d,t1,t2):
    start1,end1 = start_end_times(d,t1)
    start2,end2 = start_end_times(d,t2)
    return (start2 - end1) <= 15

# Precompute consecutive
Consecutive = {}
for d in time_slots:
    for t1 in time_slots[d]:
        for t2 in time_slots[d]:
            if t1 < t2:
                Consecutive[(d,t1,t2)] = 1 if is_consecutive(d,t1,t2) else 0

Instructors = set([ClassInfo[c][1] for c in ClassInfo])
InstructorClasses = {i:[CRN for CRN,(C,I2,P,DD,dem) in ClassInfo.items() if I2==i] for i in Instructors}

BuildingsSet = set(building_of_room(r) for r in rooms)

m = gp.Model("Minimize_Actual_Walking")

# Variables:
x = {}
for CRN,(C,I,P,DD,dem) in ClassInfo.items():
    for (d,t,pp) in FeasibleDayTime[CRN]:
        for r in FeasibleRooms[CRN]:
            x[(CRN,r,d,t)] = m.addVar(vtype=GRB.BINARY, name=f"x_{CRN}_{r}_{d}_{t}")

Z = {}
for CRN,(C,I,P,DD,dem) in ClassInfo.items():
    # For pattern classes: Z for each day in DD
    # For Lab classes: consider all days since it can choose any one day
    valid_days = DD if P!='Lab' else AllDays
    for d in valid_days:
        Z[(CRN,d)] = m.addVar(vtype=GRB.BINARY, name=f"Z_{CRN}_{d}")

y = {}
for i in Instructors:
    c_i = InstructorClasses[i]
    for d in AllDays:
        for c1 in c_i:
            for c2 in c_i:
                if c1!=c2:
                    y[(c1,c2,d)] = m.addVar(vtype=GRB.BINARY, name=f"y_{c1}_{c2}_{d}")

b_ind = {}
for CRN,(C,I,P,DD,dem) in ClassInfo.items():
    valid_days = DD if P!='Lab' else AllDays
    for d in valid_days:
        for bldg in BuildingsSet:
            b_ind[(CRN,d,bldg)] = m.addVar(vtype=GRB.BINARY, name=f"b_ind_{CRN}_{d}_{bldg}")

w_var = {}
for i in Instructors:
    c_i = InstructorClasses[i]
    for d in AllDays:
        for c1,c2 in itertools.permutations(c_i,2):
            for r1 in FeasibleRooms[c1]:
                for r2 in FeasibleRooms[c2]:
                    w_var[(c1,c2,d,r1,r2)] = m.addVar(vtype=GRB.BINARY, name=f"w_{c1}_{c2}_{d}_{r1}_{r2}")

f = {}
l = {}
for i in Instructors:
    c_i = InstructorClasses[i]
    for d in AllDays:
        for c in c_i:
            f[(c,d)] = m.addVar(vtype=GRB.BINARY, name=f"f_{c}_{d}")
            l[(c,d)] = m.addVar(vtype=GRB.BINARY, name=f"l_{c}_{d}")

break_var = {}
for i in Instructors:
    c_i = InstructorClasses[i]
    for d in AllDays:
        for c1,c2 in itertools.permutations(c_i,2):
            if c1!=c2:
                break_var[(c1,c2,d)] = m.addVar(vtype=GRB.BINARY, name=f"break_{c1}_{c2}_{d}")

m.update()

# Constraints

# Capacity is prefiltered by FeasibleRooms, but let's ensure:
for (CRN,r,d,t) in x:
    dem = ClassInfo[CRN][4]
    cap = rooms[r]['capacity']
    m.addConstr(x[(CRN,r,d,t)] <= 1*(cap>=dem))

# Assignment constraints:
for CRN,(C,I,P,DD,dem) in ClassInfo.items():
    if P=='Lab':
        # Exactly one day total
        lhs = gp.quicksum(x[(CRN,r,d,t)] for (d,t,pp) in FeasibleDayTime[CRN] for r in FeasibleRooms[CRN])
        m.addConstr(lhs == 1)
    else:
        # Each day in DD exactly one assignment
        for dd in DD:
            lhs = gp.quicksum(x[(CRN,r,dd,t)] for (d2,t,pp) in FeasibleDayTime[CRN] if d2==dd for r in FeasibleRooms[CRN])
            m.addConstr(lhs == 1)

# Pattern consistency:
# For TR: same t for T and R
# For MW: same t for M and W
# For MWF: same t for M,W,F
def pattern_days(P):
    if P=='TR':
        return ['T','R']
    elif P=='MW':
        return ['M','W']
    elif P=='MWF':
        return ['M','W','F']
    else:
        return None

for CRN,(C,I,P,DD,dem) in ClassInfo.items():
    if P in ['TR','MW','MWF']:
        p_days = pattern_days(P)
        # For these patterns, the same timeslot chosen each meeting day
        # Let’s extract feasible timeslots per day:
        day_times = {d:[t for (d2,t,pp) in FeasibleDayTime[CRN] if d2==d] for d in p_days}
        # For TR:
        # For each t in day_times of the first day, sum x(...) must equal sum x(...) of the second day same t
        # If MWF, do chain equalities.
        # We'll just link them pairwise:
        if P=='TR':
            for t_T in day_times[p_days[0]]:
                if t_T in day_times[p_days[1]]:
                    lhs_day1 = gp.quicksum(x[(CRN,r,p_days[0],t_T)] for r in FeasibleRooms[CRN])
                    lhs_day2 = gp.quicksum(x[(CRN,r,p_days[1],t_T)] for r in FeasibleRooms[CRN])
                    m.addConstr(lhs_day1 == lhs_day2)
        elif P=='MW':
            for t_M in day_times[p_days[0]]:
                if t_M in day_times[p_days[1]]:
                    lhs_M = gp.quicksum(x[(CRN,r,'M',t_M)] for r in FeasibleRooms[CRN])
                    lhs_W = gp.quicksum(x[(CRN,r,'W',t_M)] for r in FeasibleRooms[CRN])
                    m.addConstr(lhs_M == lhs_W)
        elif P=='MWF':
            # M,W,F
            # link M=W and W=F
            common_ts = set(day_times['M']).intersection(day_times['W'], day_times['F'])
            for t_m in common_ts:
                lhs_M = gp.quicksum(x[(CRN,r,'M',t_m)] for r in FeasibleRooms[CRN])
                lhs_W = gp.quicksum(x[(CRN,r,'W',t_m)] for r in FeasibleRooms[CRN])
                lhs_F = gp.quicksum(x[(CRN,r,'F',t_m)] for r in FeasibleRooms[CRN])
                m.addConstr(lhs_M == lhs_W)
                m.addConstr(lhs_W == lhs_F)

# Room availability:
for d in AllDays:
    for t in time_slots[d]:
        for rr in rooms:
            m.addConstr(gp.quicksum(x[(CRN,rr,d,t)] for CRN in ClassInfo if (CRN,rr,d,t) in x) <= 1)

# Instructor non-overlap:
for i in Instructors:
    c_i = InstructorClasses[i]
    for d in AllDays:
        for t in time_slots[d]:
            m.addConstr(gp.quicksum(x[(CRN,r,d,t)] for CRN in c_i for r in FeasibleRooms[CRN] if (CRN,r,d,t) in x) <= 1)

# Enforce consecutive scheduling to minimize breaks
for i in Instructors:
    c_i = InstructorClasses[i]
    for d in AllDays:
        for c1, c2 in itertools.permutations(c_i, 2):
            if c1 != c2:
                for t1, t2 in itertools.permutations(time_slots[d].keys(), 2):
                    if t1 < t2:  # Ensure valid time order
                        start1, end1 = start_end_times(d, t1)
                        start2, end2 = start_end_times(d, t2)
                        if start2 - end1 > 0:  # Break exists
                            m.addConstr(y[(c1, c2, d)] == 0)  # Force no gap

# Link Z:
for (CRN,(C,I,P,DD,dem)) in ClassInfo.items():
    valid_days = DD if P!='Lab' else AllDays
    for d in valid_days:
        lhs = gp.quicksum(x[(CRN,r,d,t)] for (d2,t,pp) in FeasibleDayTime[CRN] if d2==d for r in FeasibleRooms[CRN])
        m.addConstr(Z[(CRN,d)] == lhs)

# Building indicators:
for (CRN,(C,I,P,DD,dem)) in ClassInfo.items():
    valid_days = DD if P!='Lab' else AllDays
    for d in valid_days:
        # sum of b_ind over b = Z[c,d]
        m.addConstr(gp.quicksum(b_ind[(CRN,d,b)] for b in BuildingsSet) == Z[(CRN,d)])
        # b_ind[c,d,b] = sum x over rooms in building b
        for b in BuildingsSet:
            lhs = gp.quicksum(x[(CRN,r,d,t)] for (d2,t,pp) in FeasibleDayTime[CRN] if d2==d for r in FeasibleRooms[CRN] if building_of_room(r)==b)
            m.addConstr(b_ind[(CRN,d,b)] == lhs)

# Define A[c,d,t] = sum_r x(c,r,d,t)
A = {}
for CRN in ClassInfo:
    for (d,t,pp) in FeasibleDayTime[CRN]:
        A[(CRN,d,t)] = gp.quicksum(x[(CRN,r,d,t)] for r in FeasibleRooms[CRN])

for i in Instructors:
    c_i = InstructorClasses[i]
    for d in AllDays:
        for c1,c2 in itertools.permutations(c_i,2):
            if c1!=c2:
                Ytt_list = []
                # Filter timeslots to only those in FeasibleDayTime for c1 and c2:
                feasible_c1 = [(d2,t2) for (d2,t2,pp) in FeasibleDayTime[c1] if d2==d]
                feasible_c2 = [(d3,t3) for (d3,t3,pp) in FeasibleDayTime[c2] if d3==d]
                
                # Extract just timeslot indices for convenience
                c1_ts = [t1 for (dd,t1) in feasible_c1]
                c2_ts = [t2 for (dd,t2) in feasible_c2]

                for t1 in c1_ts:
                    for t2 in c2_ts:
                        if t1 < t2 and (d,t1,t2) in Consecutive:
                            # Now we know A[(c1,d,t1)] and A[(c2,d,t2)] exist
                            Ytt_var = m.addVar(vtype=GRB.BINARY, name=f"Ytt_{c1}_{c2}_{d}_{t1}_{t2}")
                            m.addConstr(Ytt_var <= A[(c1,d,t1)])
                            m.addConstr(Ytt_var <= A[(c2,d,t2)])
                            if Consecutive[(d,t1,t2)]==0:
                                m.addConstr(Ytt_var <= 0)
                            m.addConstr(Ytt_var >= A[(c1,d,t1)]+A[(c2,d,t2)]+Consecutive[(d,t1,t2)]-2)
                            Ytt_list.append(Ytt_var)
                m.addConstr(y[(c1,c2,d)] <= gp.quicksum(Ytt_list))
                m.addConstr(gp.quicksum(Ytt_list) >= y[(c1,c2,d)])



for i in Instructors:
    c_i = InstructorClasses[i]
    for c in c_i:
        (CRN,I,P,DD,dem) = ClassInfo[c]
        # Only loop over the valid days for this class:
        valid_days = DD if P!='Lab' else AllDays
        for d in valid_days:
            # Now Z[(c,d)] is guaranteed to exist
            m.addConstr(f[(c,d)] <= Z[(c,d)])
            m.addConstr(f[(c,d)] + gp.quicksum(y[(c2,c,d)] for c2 in c_i if c2!=c) <= 1)
            m.addConstr(l[(c,d)] <= Z[(c,d)])
            m.addConstr(l[(c,d)] + gp.quicksum(y[(c,c2,d)] for c2 in c_i if c2!=c) <= 1)

        # Also ensure sum of f and l per day:
        # If you need to ensure at most one first and last class per day per instructor:
        # The instructor might not have classes every day, so only sum over valid days
        for d in AllDays:
            # If no class meets that day for that instructor, f[(c,d)] and l[(c,d)] may not be relevant.
            # To handle this gracefully, you can either skip or ensure that f and l were created only for valid days.
            pass

    # sum f_{c,d} ≤ 1 if classes that day, sum l_{c,d} ≤ 1 if classes that day
    # If no class that day, Z sum is 0, automatically f and l can be 0.
    for d in AllDays:
        m.addConstr(gp.quicksum(f[(c,d)] for c in c_i) <= 1)
        m.addConstr(gp.quicksum(l[(c,d)] for c in c_i) <= 1)

# w_var linking:
for (c1,c2,d,r1,r2) in w_var:
    # w_var ≤ sum_t x[c1,r1,d,t] and ≤ sum_t x[c2,r2,d,t]
    lhs_c1 = gp.quicksum(x[(c1,r1,d,t)] for (d2,t,pp) in FeasibleDayTime[c1] if d2==d)
    lhs_c2 = gp.quicksum(x[(c2,r2,d,t)] for (d2,t,pp) in FeasibleDayTime[c2] if d2==d)
    m.addConstr(w_var[(c1,c2,d,r1,r2)] <= lhs_c1)
    m.addConstr(w_var[(c1,c2,d,r1,r2)] <= lhs_c2)

# break_var for non-consecutive:
for i in Instructors:
    c_i = InstructorClasses[i]
    for d in AllDays:
        for c1,c2 in itertools.permutations(c_i,2):
            if c1!=c2:
                # break_var ≤ Z[c1,d], break_var ≤ Z[c2,d]
                # break_var ≤ 2 - (y[c1,c2,d]+y[c2,c1,d])
                m.addConstr(break_var[(c1,c2,d)] <= Z[(c1,d)] if (c1,d) in Z else break_var[(c1,c2,d)]<=0)
                m.addConstr(break_var[(c1,c2,d)] <= Z[(c2,d)] if (c2,d) in Z else break_var[(c1,c2,d)]<=0)
                m.addConstr(break_var[(c1,c2,d)] <= 2-(y[(c1,c2,d)]+y[(c2,c1,d)]))
                # break_var≥Z[c1,d]+Z[c2,d]- (y_{c1,c2,d}+y_{c2,c1,d}) -1
                zc1 = Z[(c1,d)] if (c1,d) in Z else 0
                zc2 = Z[(c2,d)] if (c2,d) in Z else 0
                m.addConstr(break_var[(c1,c2,d)] >= zc1+zc2-(y[(c1,c2,d)]+y[(c2,c1,d)])-1)

# For break cost calculation we need brw_{c1,c2,d,b1,b2}
brw = {}
for i in Instructors:
    c_i = InstructorClasses[i]
    for d in AllDays:
        for c1,c2 in itertools.permutations(c_i,2):
            if c1!=c2:
                for b1 in BuildingsSet:
                    for b2 in BuildingsSet:
                        brw[(c1,c2,d,b1,b2)] = m.addVar(vtype=GRB.BINARY, name=f"brw_{c1}_{c2}_{d}_{b1}_{b2}")

m.update()
# brw ≤ break_var, brw ≤ b_ind[c1,d,b1], brw ≤ b_ind[c2,d,b2]
for i in Instructors:
    c_i = InstructorClasses[i]
    for d in AllDays:
        for c1,c2 in itertools.permutations(c_i,2):
            if c1!=c2:
                for b1 in BuildingsSet:
                    for b2 in BuildingsSet:
                        if (c1,d,b1) in b_ind and (c2,d,b2) in b_ind:
                            m.addConstr(brw[(c1,c2,d,b1,b2)] <= break_var[(c1,c2,d)])
                            m.addConstr(brw[(c1,c2,d,b1,b2)] <= b_ind[(c1,d,b1)])
                            m.addConstr(brw[(c1,c2,d,b1,b2)] <= b_ind[(c2,d,b2)])
                            m.addConstr(brw[(c1,c2,d,b1,b2)] >= b_ind[(c1,d,b1)]+b_ind[(c2,d,b2)]+break_var[(c1,c2,d)]-2)
                        else:
                            # If not feasible, force to 0
                            m.addConstr(brw[(c1,c2,d,b1,b2)] == 0)

# Also ensure w_var ≤ y for consecutive transitions:
for (c1,c2,d,r1,r2) in w_var:
    m.addConstr(w_var[(c1,c2,d,r1,r2)] <= y[(c1,c2,d)])

# Objective:
obj_expr = gp.LinExpr()


for i in Instructors:
    c_i = InstructorClasses[i]
    for c in c_i:
        (CRN,I,P,DD,dem) = ClassInfo[c]
        valid_days = DD if P!='Lab' else AllDays
        for d in valid_days:  # Only iterate over valid_days!
            # Now we know Z[(c,d)] and b_ind[(c,d,b)] exist
            for b in BuildingsSet:
                # create fh etc. 
                fh = m.addVar(vtype=GRB.BINARY, name=f"fh_{c}_{d}_{b}")
                m.addConstr(fh <= f[(c,d)])
                m.addConstr(fh <= b_ind[(c,d,b)])  # Safe now since (c,d,b) exists
                m.addConstr(fh >= f[(c,d)]+b_ind[(c,d,b)]-1)
                office_to_c_cost = cost_between_buildings(OfficeBuilding,b,OfficeBuilding,W_intra,W_inter)
                obj_expr += office_to_c_cost * fh

# consecutive classes cost:
for (c1,c2,d,r1,r2) in w_var:
    b1 = building_of_room(r1)
    b2 = building_of_room(r2)
    trans_cost = cost_between_buildings(b1,b2,OfficeBuilding,W_intra,W_inter)
    building_transition_penalty = 15  # Additional penalty for switching buildings
    if b1 != b2:
        trans_cost += building_transition_penalty
    obj_expr += trans_cost * w_var[(c1, c2, d, r1, r2)]
    

# break cost:
for i in Instructors:
    c_i = InstructorClasses[i]
    for d in AllDays:
        for c1,c2 in itertools.permutations(c_i,2):
            if c1!=c2:
                for b1 in BuildingsSet:
                    for b2 in BuildingsSet:
                        return_office_cost = cost_between_buildings(b1,OfficeBuilding,OfficeBuilding,W_intra,W_inter) \
                                             + cost_between_buildings(OfficeBuilding,b2,OfficeBuilding,W_intra,W_inter)
                        break_weight = 20  # Increase weight for break costs
                        obj_expr += break_weight * return_office_cost * brw[(c1, c2, d, b1, b2)]

for i in Instructors:
    c_i = InstructorClasses[i]
    for c in c_i:
        (CRN,I,P,DD,dem) = ClassInfo[c]
        valid_days = DD if P!='Lab' else AllDays
        for d in valid_days:  # Only iterate over days this class meets
            for b in BuildingsSet:
                # Now safe to reference b_ind[(c,d,b)]
                lh = m.addVar(vtype=GRB.BINARY, name=f"lh_{c}_{d}_{b}")
                m.addConstr(lh <= l[(c,d)])
                m.addConstr(lh <= b_ind[(c,d,b)])
                m.addConstr(lh >= l[(c,d)]+b_ind[(c,d,b)]-1)
                off_cost = cost_between_buildings(b,OfficeBuilding,OfficeBuilding,W_intra,W_inter)
                obj_expr += off_cost * lh

m.setObjective(obj_expr, GRB.MINIMIZE)

m.optimize()

if m.status == GRB.OPTIMAL:
    print("Optimal solution found with objective:", m.objVal)
    
    # Extract chosen assignments
    chosen = []
    for (CRN,r,d,t) in x:
        if x[(CRN,r,d,t)].X > 0.5:
            (C,I,P,DD,dem) = ClassInfo[CRN]
            chosen.append((I, d, t, CRN, r, P))
    
    # We'll need to parse start times to sort classes in chronological order
    def slot_start_minutes(slot_str):
        # Convert 'H:MM AM - H:MM AM' to start time in minutes
        match = re.match(r'(\d+):(\d+) (\wM)', slot_str)
        hour = int(match.group(1))
        minute = int(match.group(2))
        ampm = match.group(3)
        if ampm == 'PM' and hour < 12:
            hour += 12
        return hour*60 + minute
    
    # Create a schedule structure: {instructor: {day: [(start_min, CRN, room, Pattern, slot_str, building)]}}
    schedule = {}
    for (I,d,t,CRN,r,P) in chosen:
        slot_str = time_slots[d][t]
        start_min = slot_start_minutes(slot_str)
        bldg = building_of_room(r)
        if I not in schedule:
            schedule[I] = {}
        if d not in schedule[I]:
            schedule[I][d] = []
        schedule[I][d].append((start_min, CRN, r, P, slot_str, bldg))
    
    # Sort each day by start time
    for I in schedule:
        for d in schedule[I]:
            schedule[I][d].sort(key=lambda x: x[0])
    
    # Print the schedule in a neat format
    for I in sorted(schedule.keys()):
        print(f"\nSchedule for Instructor: {I}")
        for d in ['M','T','W','R','F']:
            if d in schedule[I]:
                print(f"  {d}:")
                for (start_min, CRN, r, P, slot_str, bldg) in schedule[I][d]:
                    print(f"    {slot_str} -> CRN {CRN} ({P}), Room {r}, Building {bldg}")

In [ ]:
if m.status == GRB.OPTIMAL:
    print("Optimal solution found with objective:", m.objVal)
    
    total_cost = 0
    first_last_cost = 0
    consecutive_cost = 0
    break_cost = 0
    
    # First/Last Class Costs
    print("\nCost for First/Last Classes:")
    for i in Instructors:
        c_i = InstructorClasses[i]
        for d in AllDays:
            # First Class
            for c in c_i:
                for b in BuildingsSet:
                    if f[(c, d)].X > 0.5 and b_ind[(c, d, b)].X > 0.5:
                        cost_to_office = cost_between_buildings(OfficeBuilding, b, OfficeBuilding, W_intra, W_inter)
                        print(f"Instructor {i}, Day {d}, First Class in Building {b}: Cost {cost_to_office}")
                        first_last_cost += cost_to_office
            
            # Last Class
            for c in c_i:
                for b in BuildingsSet:
                    if l[(c, d)].X > 0.5 and b_ind[(c, d, b)].X > 0.5:
                        cost_from_office = cost_between_buildings(b, OfficeBuilding, OfficeBuilding, W_intra, W_inter)
                        print(f"Instructor {i}, Day {d}, Last Class in Building {b}: Cost {cost_from_office}")
                        first_last_cost += cost_from_office
    
    # Consecutive Class Costs
    print("\nCost for Consecutive Classes:")
    for (c1, c2, d, r1, r2) in w_var:
        if w_var[(c1, c2, d, r1, r2)].X > 0.5:
            b1 = building_of_room(r1)
            b2 = building_of_room(r2)
            cost_between_classes = cost_between_buildings(b1, b2, OfficeBuilding, W_intra, W_inter)
            print(f"Instructor Transition from {c1} in {b1} to {c2} in {b2} on Day {d}: Cost {cost_between_classes}")
            consecutive_cost += cost_between_classes
    
    # Break Costs
    print("\nCost for Breaks:")
    for i in Instructors:
        c_i = InstructorClasses[i]
        for d in AllDays:
            for c1, c2 in itertools.permutations(c_i, 2):
                if break_var[(c1, c2, d)].X > 0.5:
                    for b1 in BuildingsSet:
                        for b2 in BuildingsSet:
                            if brw[(c1, c2, d, b1, b2)].X > 0.5:
                                return_to_office_cost = cost_between_buildings(b1, OfficeBuilding, OfficeBuilding, W_intra, W_inter) \
                                                        + cost_between_buildings(OfficeBuilding, b2, OfficeBuilding, W_intra, W_inter)
                                print(f"Instructor {i}, Break from {c1} in {b1} to {c2} in {b2} on Day {d}: Cost {return_to_office_cost}")
                                break_cost += return_to_office_cost
    
    total_cost = first_last_cost + consecutive_cost + break_cost
    print("\n--- Summary of Costs ---")
    print(f"First/Last Class Cost: {first_last_cost}")
    print(f"Consecutive Class Cost: {consecutive_cost}")
    print(f"Break Cost: {break_cost}")
    print(f"Total Calculated Cost: {total_cost}")


In [ ]:
#walking distance restriction

In [ ]:
import gurobipy as gp
from gurobipy import GRB
import re
import itertools

# -----------------------------
# INPUT DATA
# -----------------------------
courses = {
    'Owen Tyler Dodd': [
        {'CRN': 30522, 'days': ['T', 'R'], 'demand': 41}
    ],
    'Rui Zhu': [
        {'CRN': 30524, 'days': ['M', 'W'], 'demand': 44}
    ],
    'Talayeh Razzaghi': [
        {'CRN': 43516, 'days': ['T', 'R'], 'demand': 17}
    ],
    'Theodore B Trafalis': [
        {'CRN': 41627, 'days': ['T', 'R'], 'demand': 9}
    ],
    'Shivakumar Raman': [
        {'CRN': 30529, 'days': ['T', 'R'], 'demand': 44},
        {'CRN': 30530, 'days': 'Lab', 'demand': 16},
        {'CRN': 30532, 'days': 'Lab', 'demand': 16},
        {'CRN': 30533, 'days': 'Lab', 'demand': 8},
        {'CRN': 34447, 'days': 'Lab', 'demand': 5},
        {'CRN': 42741, 'days': 'Lab', 'demand': 4}
    ],
    'Charles D Nicholson': [
        {'CRN': 30535, 'days': ['T', 'R'], 'demand': 46},
        {'CRN': 34608, 'days': ['T', 'R'], 'demand': 31}
    ],
    'Cesar Alexander Ruiz Torres': [
        {'CRN': 30536, 'days': ['T', 'R'], 'demand': 39}
    ],
    'Kash A Barker': [
        {'CRN': 32173, 'days': ['M', 'W', 'F'], 'demand': 40},
        {'CRN': 30539, 'days': ['M', 'W'], 'demand': 65}
    ],
    'Andres D Gonzalez': [
        {'CRN': 30541, 'days': ['T', 'R'], 'demand': 70},
        {'CRN': 30561, 'days': ['T', 'R'], 'demand': 20}
    ],
    'Ziho Kang': [
        {'CRN': 30544, 'days': ['M', 'W'], 'demand': 39},
        {'CRN': 30548, 'days': 'Lab', 'demand': 15},
        {'CRN': 30550, 'days': 'Lab', 'demand': 13},
        {'CRN': 42308, 'days': 'Lab', 'demand': 19},
        {'CRN': 30556, 'days': ['M', 'W', 'F'], 'demand': 45},
        {'CRN': 30562, 'days': ['M', 'W'], 'demand': 10}
    ]
}

rooms = {
    '324': {'capacity': 10, 'building': 'BL'},
    '117': {'capacity': 55, 'building': 'CEC'},
    '121': {'capacity': 25, 'building': 'CEC'},
    '205': {'capacity': 20, 'building': 'CEC'},
    '438': {'capacity': 25, 'building': 'CEC'},
    'S0014': {'capacity': 20, 'building': 'CEC'},
    '300': {'capacity': 100, 'building': 'FH'},
    '304': {'capacity': 100, 'building': 'FH'},
    '336': {'capacity': 100, 'building': 'FH'},
    '123': {'capacity': 120, 'building': 'GLCH'},
    '127': {'capacity': 130, 'building': 'GLG'},
    '108': {'capacity': 80, 'building': 'PHSC'},
    'M0204': {'capacity': 70, 'building': 'SEC'},
    'N0202': {'capacity': 80, 'building': 'SEC'},
    'P0201': {'capacity': 50, 'building': 'SEC'}
}

def get_pattern(days):
    if days == 'Lab':
        return 'Lab'
    dset = set(days)
    if dset == {'T','R'}:
        return 'TR'
    elif dset == {'M','W'}:
        return 'MW'
    elif dset == {'M','W','F'}:
        return 'MWF'
    else:
        return 'Lab'

AllDays = ['M','T','W','R','F']

time_slots = {
    'M': {0: '9:30 AM - 10:20 AM',1:'10:30 AM - 11:20 AM',2:'11:30 AM - 12:20 PM',3:'1:30 PM - 2:45 PM',4:'3:00 PM - 4:15 PM'},
    'T': {0:'7:30 AM - 8:45 AM',1:'9:00 AM - 10:15 AM',2:'10:30 AM - 11:45 AM',3:'12:00 PM - 1:15 PM',4:'1:30 PM - 2:45 PM',5:'3:00 PM - 4:15 PM',6:'4:30 PM - 5:45 PM'},
    'W': {0:'9:30 AM - 10:20 AM',1:'10:30 AM - 11:20 AM',2:'11:30 AM - 12:20 PM',3:'1:30 PM - 2:45 PM',4:'3:00 PM - 4:15 PM'},
    'R': {0:'7:30 AM - 8:45 AM',1:'9:00 AM - 10:15 AM',2:'10:30 AM - 11:45 AM',3:'12:00 PM - 1:15 PM',4:'1:30 PM - 2:45 PM',5:'3:00 PM - 4:15 PM',6:'4:30 PM - 5:45 PM'},
    'F': {0:'9:30 AM - 10:20 AM',1:'10:30 AM - 11:20 AM',2:'11:30 AM - 12:20 PM'}
}

# Add distinct lab slot for each day
time_slots['M'][5] = '4:30 PM - 5:30 PM (Lab)'
time_slots['T'][7] = '4:30 PM - 5:30 PM (Lab)'
time_slots['W'][5] = '4:30 PM - 5:30 PM (Lab)'
time_slots['R'][7] = '4:30 PM - 5:30 PM (Lab)'
time_slots['F'][3] = '4:30 PM - 5:30 PM (Lab)'

lab_slot = {'M':5,'T':7,'W':5,'R':7,'F':3}

Classes = []
for instr,clist in courses.items():
    clean_instr = instr.strip()
    for c in clist:
        CRN = c['CRN']
        days = c['days']
        dem = c['demand']
        pattern = get_pattern(days)
        DD = tuple(days) if pattern!='Lab' else tuple(AllDays)
        Classes.append((CRN, clean_instr, pattern, DD, dem))

ClassInfo = {c[0]: c for c in Classes}
FeasibleRooms = {CRN: [r for r in rooms if rooms[r]['capacity']>=ClassInfo[CRN][4]] for CRN in ClassInfo}

# FeasibleDayTime calculation:
FeasibleDayTime = {}
for CRN,(C,I,P,DD,dem) in ClassInfo.items():
    feasible = []
    if P=='Lab':
        # Only lab slots each day
        for d in AllDays:
            feasible.append((d, lab_slot[d], P))
    else:
        for d in DD:
            for t_idx in time_slots[d].keys():
                if t_idx != lab_slot[d]:
                    feasible.append((d,t_idx,P))
    FeasibleDayTime[CRN] = feasible

OfficeBuilding = "CEC"
W_intra = 7
W_inter = 12

def building_of_room(r):
    return rooms[r]['building']

def cost_between_buildings(b1, b2, OfficeBuilding="CEC", W_intra=7, W_inter=12):
    if b1 == b2:
        return W_intra
    if b1==OfficeBuilding or b2==OfficeBuilding:
        if b1!=b2:
            return W_inter
        else:
            return W_intra
    return W_inter

# Convert timeslot strings to start and end times in minutes
def parse_time_in_minutes(time_str):
    # e.g. "9:30 AM - 10:20 AM"
    pattern = r'(\d+):(\d+) (AM|PM) - (\d+):(\d+) (AM|PM)'
    m = re.match(pattern, time_str)
    if not m:
        # For lab and others if needed
        # e.g. "4:30 PM - 5:30 PM (Lab)"
        pattern_lab = r'(\d+):(\d+) (AM|PM) - (\d+):(\d+) (AM|PM)'
        m_lab = re.match(pattern_lab, time_str)
        if not m_lab:
            raise ValueError("Time format not recognized: "+time_str)
        start_h, start_min, start_ampm, end_h, end_min, end_ampm = m_lab.groups()
    else:
        start_h, start_min, start_ampm, end_h, end_min, end_ampm = m.groups()

    start_h = int(start_h)
    end_h = int(end_h)
    start_min = int(start_min)
    end_min = int(end_min)

    if start_ampm == 'PM' and start_h < 12:
        start_h += 12
    if end_ampm == 'PM' and end_h < 12:
        end_h += 12

    start_total = start_h * 60 + start_min
    end_total = end_h * 60 + end_min
    return start_total, end_total

# Dictionary to store start/end times in minutes
times_in_minutes = {}
for d in time_slots:
    times_in_minutes[d] = {}
    for t in time_slots[d]:
        start_m, end_m = parse_time_in_minutes(time_slots[d][t])
        times_in_minutes[d][t] = (start_m, end_m)

# Define consecutive: now only if t2 == t1+1
Consecutive = {}
for d in time_slots:
    sorted_ts = sorted(time_slots[d].keys())
    for t1 in sorted_ts:
        for t2 in sorted_ts:
            Consecutive[(d,t1,t2)] = 1 if t2 == t1+1 else 0

Instructors = set([ClassInfo[c][1] for c in ClassInfo])
InstructorClasses = {i:[CRN for CRN,(C,I2,P,DD,dem) in ClassInfo.items() if I2==i] for i in Instructors}

BuildingsSet = set(building_of_room(r) for r in rooms)

m = gp.Model("Minimize_Actual_Walking")

x = {}
for CRN,(C,I,P,DD,dem) in ClassInfo.items():
    for (d,t,pp) in FeasibleDayTime[CRN]:
        for r in FeasibleRooms[CRN]:
            x[(CRN,r,d,t)] = m.addVar(vtype=GRB.BINARY, name=f"x_{CRN}_{r}_{d}_{t}")

Z = {}
for CRN,(C,I,P,DD,dem) in ClassInfo.items():
    valid_days = DD if P!='Lab' else AllDays
    for d in valid_days:
        Z[(CRN,d)] = m.addVar(vtype=GRB.BINARY, name=f"Z_{CRN}_{d}")

y = {}
for i in Instructors:
    c_i = InstructorClasses[i]
    for d in AllDays:
        for c1 in c_i:
            for c2 in c_i:
                if c1!=c2:
                    y[(c1,c2,d)] = m.addVar(vtype=GRB.BINARY, name=f"y_{c1}_{c2}_{d}")

b_ind = {}
for CRN,(C,I,P,DD,dem) in ClassInfo.items():
    valid_days = DD if P!='Lab' else AllDays
    for d in valid_days:
        for bldg in BuildingsSet:
            b_ind[(CRN,d,bldg)] = m.addVar(vtype=GRB.BINARY, name=f"b_ind_{CRN}_{d}_{bldg}")

w_var = {}
for i in Instructors:
    c_i = InstructorClasses[i]
    for d in AllDays:
        for c1,c2 in itertools.permutations(c_i,2):
            for r1 in FeasibleRooms[c1]:
                for r2 in FeasibleRooms[c2]:
                    w_var[(c1,c2,d,r1,r2)] = m.addVar(vtype=GRB.BINARY, name=f"w_{c1}_{c2}_{d}_{r1}_{r2}")

f = {}
l = {}
for i in Instructors:
    c_i = InstructorClasses[i]
    for d in AllDays:
        for c in c_i:
            f[(c,d)] = m.addVar(vtype=GRB.BINARY, name=f"f_{c}_{d}")
            l[(c,d)] = m.addVar(vtype=GRB.BINARY, name=f"l_{c}_{d}")

break_var = {}
for i in Instructors:
    c_i = InstructorClasses[i]
    for d in AllDays:
        for c1,c2 in itertools.permutations(c_i,2):
            if c1!=c2:
                break_var[(c1,c2,d)] = m.addVar(vtype=GRB.BINARY, name=f"break_{c1}_{c2}_{d}")


m.update()

# Capacity
for (CRN,r,d,t) in x:
    dem = ClassInfo[CRN][4]
    cap = rooms[r]['capacity']
    m.addConstr(x[(CRN,r,d,t)] <= 1*(cap>=dem))

# Assignment constraints
for CRN,(C,I,P,DD,dem) in ClassInfo.items():
    if P=='Lab':
        lhs = gp.quicksum(x[(CRN,r,d,t)] for (d,t,pp) in FeasibleDayTime[CRN] for r in FeasibleRooms[CRN])
        m.addConstr(lhs == 1)
    else:
        for dd in DD:
            lhs = gp.quicksum(x[(CRN,r,dd,t)] for (d2,t,pp) in FeasibleDayTime[CRN] if d2==dd for r in FeasibleRooms[CRN])
            m.addConstr(lhs == 1)

# Pattern consistency
def pattern_days(P):
    if P=='TR':
        return ['T','R']
    elif P=='MW':
        return ['M','W']
    elif P=='MWF':
        return ['M','W','F']
    else:
        return None

for CRN,(C,I,P,DD,dem) in ClassInfo.items():
    if P in ['TR','MW','MWF']:
        p_days = pattern_days(P)
        day_times = {d:[t for (d2,t,pp) in FeasibleDayTime[CRN] if d2==d] for d in p_days}
        if P=='TR':
            for t_T in day_times[p_days[0]]:
                if t_T in day_times[p_days[1]]:
                    lhs_day1 = gp.quicksum(x[(CRN,r,p_days[0],t_T)] for r in FeasibleRooms[CRN])
                    lhs_day2 = gp.quicksum(x[(CRN,r,p_days[1],t_T)] for r in FeasibleRooms[CRN])
                    m.addConstr(lhs_day1 == lhs_day2)
        elif P=='MW':
            for t_M in day_times[p_days[0]]:
                if t_M in day_times[p_days[1]]:
                    lhs_M = gp.quicksum(x[(CRN,r,'M',t_M)] for r in FeasibleRooms[CRN])
                    lhs_W = gp.quicksum(x[(CRN,r,'W',t_M)] for r in FeasibleRooms[CRN])
                    m.addConstr(lhs_M == lhs_W)
        elif P=='MWF':
            common_ts = set(day_times['M']).intersection(day_times['W'], day_times['F'])
            for t_m in common_ts:
                lhs_M = gp.quicksum(x[(CRN,r,'M',t_m)] for r in FeasibleRooms[CRN])
                lhs_W = gp.quicksum(x[(CRN,r,'W',t_m)] for r in FeasibleRooms[CRN])
                lhs_F = gp.quicksum(x[(CRN,r,'F',t_m)] for r in FeasibleRooms[CRN])
                m.addConstr(lhs_M == lhs_W)
                m.addConstr(lhs_W == lhs_F)

# Room availability
for d in AllDays:
    for t in time_slots[d]:
        m.addConstr(gp.quicksum(x[(CRN,rr,d,t)] for CRN in ClassInfo for rr in FeasibleRooms[CRN] if (CRN,rr,d,t) in x) <= 1)

# Overlapping Time Slots: Explicit Mapping
overlapping_slots = {}
for d in time_slots:
    overlapping_slots[d] = []
    for t1, t2 in itertools.combinations(time_slots[d].keys(), 2):
        start_1, end_1 = times_in_minutes[d][t1]
        start_2, end_2 = times_in_minutes[d][t2]
        if not (end_1 <= start_2 or end_2 <= start_1): # overlapping if intervals intersect
            overlapping_slots[d].append((t1, t2))

# Instructor non-overlap with overlapping time slots
for i in Instructors:
    c_i = InstructorClasses[i]
    for d in AllDays:
        # Prevent assignments to the same slot
        for t in time_slots[d]:
            m.addConstr(
                gp.quicksum(
                    x[(CRN, r, d, t)] for CRN in c_i for r in FeasibleRooms[CRN] if (CRN, r, d, t) in x
                ) <= 1
            )
        
        # Prevent assignments to overlapping slots
        for t1, t2 in overlapping_slots[d]:
            m.addConstr(
                gp.quicksum(
                    x[(CRN, r, d, tm)] for CRN in c_i for r in FeasibleRooms[CRN] for tm in [t1, t2]
                    if (CRN, r, d, tm) in x
                ) <= 1
            )

# Z linking
for (CRN,(C,I,P,DD,dem)) in ClassInfo.items():
    valid_days = DD if P!='Lab' else AllDays
    for d in valid_days:
        lhs = gp.quicksum(x[(CRN,r,d,t)] for (d2,t,pp) in FeasibleDayTime[CRN] if d2==d for r in FeasibleRooms[CRN])
        m.addConstr(Z[(CRN,d)] == lhs)

# Building indicators
for (CRN,(C,I,P,DD,dem)) in ClassInfo.items():
    valid_days = DD if P!='Lab' else AllDays
    for d in valid_days:
        m.addConstr(gp.quicksum(b_ind[(CRN,d,b)] for b in BuildingsSet) == Z[(CRN,d)])
        for b in BuildingsSet:
            lhs = gp.quicksum(x[(CRN,r,d,t)] for (d2,t,pp) in FeasibleDayTime[CRN] if d2==d for r in FeasibleRooms[CRN] if building_of_room(r)==b)
            m.addConstr(b_ind[(CRN,d,b)] == lhs)

# A[c,d,t]
A = {}
for CRN in ClassInfo:
    for (d,t,pp) in FeasibleDayTime[CRN]:
        A[(CRN,d,t)] = gp.quicksum(x[(CRN,r,d,t)] for r in FeasibleRooms[CRN])

# Consecutive logic:
for i in Instructors:
    c_i = InstructorClasses[i]
    for d in AllDays:
        for c1,c2 in itertools.permutations(c_i,2):
            if c1!=c2:
                Ytt_list = []
                feasible_c1 = [(d2,t2) for (d2,t2,pp) in FeasibleDayTime[c1] if d2==d]
                feasible_c2 = [(d3,t3) for (d3,t3,pp) in FeasibleDayTime[c2] if d3==d]
                c1_ts = [t1 for (dd,t1) in feasible_c1]
                c2_ts = [t2 for (dd,t2) in feasible_c2]

                for t1 in c1_ts:
                    for t2 in c2_ts:
                        if (d,t1,t2) in Consecutive:
                            if Consecutive[(d,t1,t2)]==1:
                                Ytt_var = m.addVar(vtype=GRB.BINARY, name=f"Ytt_{c1}_{c2}_{d}_{t1}_{t2}")
                                m.addConstr(Ytt_var <= A[(c1,d,t1)])
                                m.addConstr(Ytt_var <= A[(c2,d,t2)])
                                m.addConstr(Ytt_var >= A[(c1,d,t1)]+A[(c2,d,t2)]-1)
                                Ytt_list.append(Ytt_var)
                m.addConstr(y[(c1,c2,d)] <= gp.quicksum(Ytt_list))
                m.addConstr(gp.quicksum(Ytt_list) >= y[(c1,c2,d)])

for i in Instructors:
    c_i = InstructorClasses[i]
    for c in c_i:
        (CRN,I,P,DD,dem) = ClassInfo[c]
        valid_days = DD if P!='Lab' else AllDays
        for d in valid_days:
            m.addConstr(f[(c,d)] <= Z[(c,d)])
            m.addConstr(f[(c,d)] + gp.quicksum(y[(c2,c,d)] for c2 in c_i if c2!=c) <= 1)
            m.addConstr(l[(c,d)] <= Z[(c,d)])
            m.addConstr(l[(c,d)] + gp.quicksum(y[(c,c2,d)] for c2 in c_i if c2!=c) <= 1)

    for d in AllDays:
        m.addConstr(gp.quicksum(f[(c,d)] for c in c_i if (c,d) in f) <= 1)
        m.addConstr(gp.quicksum(l[(c,d)] for c in c_i if (c,d) in l) <= 1)


for (c1,c2,d,r1,r2) in w_var:
    lhs_c1 = gp.quicksum(x[(c1,r1,d,t)] for (d2,t,pp) in FeasibleDayTime[c1] if d2==d)
    lhs_c2 = gp.quicksum(x[(c2,r2,d,t)] for (d2,t,pp) in FeasibleDayTime[c2] if d2==d)
    m.addConstr(w_var[(c1,c2,d,r1,r2)] <= lhs_c1)
    m.addConstr(w_var[(c1,c2,d,r1,r2)] <= lhs_c2)

for i in Instructors:
    c_i = InstructorClasses[i]
    for d in AllDays:
        for c1,c2 in itertools.permutations(c_i,2):
            if c1!=c2:
                if (c1,d) in Z and (c2,d) in Z:
                    m.addConstr(break_var[(c1, c2, d)] <= Z[(c1, d)])
                    m.addConstr(break_var[(c1, c2, d)] <= Z[(c2, d)])
                    m.addConstr(break_var[(c1, c2, d)] <= 2 - (y[(c1, c2, d)] + y[(c2, c1, d)]))
                    m.addConstr(
                        break_var[(c1, c2, d)] >= Z[(c1, d)] + Z[(c2, d)] - (y[(c1, c2, d)] + y[(c2, c1, d)]) - 1
                    )
                else:
                    m.addConstr(break_var[(c1, c2, d)] == 0)

brw = {}
for i in Instructors:
    c_i = InstructorClasses[i]
    for d in AllDays:
        for c1,c2 in itertools.permutations(c_i,2):
            if c1!=c2:
                for b1 in BuildingsSet:
                    for b2 in BuildingsSet:
                        brw[(c1,c2,d,b1,b2)] = m.addVar(vtype=GRB.BINARY, name=f"brw_{c1}_{c2}_{d}_{b1}_{b2}")

m.update()

for i in Instructors:
    c_i = InstructorClasses[i]
    for d in AllDays:
        for c1,c2 in itertools.permutations(c_i,2):
            if c1!=c2:
                for b1 in BuildingsSet:
                    for b2 in BuildingsSet:
                        if (c1,d,b1) in b_ind and (c2,d,b2) in b_ind:
                            m.addConstr(brw[(c1,c2,d,b1,b2)] <= break_var[(c1,c2,d)])
                            m.addConstr(brw[(c1,c2,d,b1,b2)] <= b_ind[(c1,d,b1)])
                            m.addConstr(brw[(c1,c2,d,b1,b2)] <= b_ind[(c2,d,b2)])
                            m.addConstr(brw[(c1,c2,d,b1,b2)] >= b_ind[(c1,d,b1)]+b_ind[(c2,d,b2)]+break_var[(c1,c2,d)]-2)
                        else:
                            m.addConstr(brw[(c1,c2,d,b1,b2)] == 0)

for (c1,c2,d,r1,r2) in w_var:
    m.addConstr(w_var[(c1,c2,d,r1,r2)] <= y[(c1,c2,d)])


obj_expr = gp.LinExpr()

for i in Instructors:
    c_i = InstructorClasses[i]
    for c in c_i:
        (CRN,I,P,DD,dem) = ClassInfo[c]
        valid_days = DD if P!='Lab' else AllDays
        for d in valid_days:
            for b in BuildingsSet:
                fh = m.addVar(vtype=GRB.BINARY)
                m.addConstr(fh <= f[(c,d)])
                m.addConstr(fh <= b_ind[(c,d,b)])
                m.addConstr(fh >= f[(c,d)]+b_ind[(c,d,b)]-1)
                office_to_c_cost = cost_between_buildings(OfficeBuilding,b,OfficeBuilding,W_intra,W_inter)
                obj_expr += office_to_c_cost * fh

for (c1,c2,d,r1,r2) in w_var:
    b1 = building_of_room(r1)
    b2 = building_of_room(r2)
    trans_cost = cost_between_buildings(b1,b2,OfficeBuilding,W_intra,W_inter)
    obj_expr += trans_cost * w_var[(c1, c2, d, r1, r2)]

for i in Instructors:
    c_i = InstructorClasses[i]
    for d in AllDays:
        for c1,c2 in itertools.permutations(c_i,2):
            if c1!=c2:
                for b1 in BuildingsSet:
                    for b2 in BuildingsSet:
                        return_office_cost = cost_between_buildings(b1,OfficeBuilding,OfficeBuilding,W_intra,W_inter) \
                                             + cost_between_buildings(OfficeBuilding,b2,OfficeBuilding,W_intra,W_inter)
                        obj_expr += return_office_cost * brw[(c1, c2, d, b1, b2)]

for i in Instructors:
    c_i = InstructorClasses[i]
    for c in c_i:
        (CRN,I,P,DD,dem) = ClassInfo[c]
        valid_days = DD if P!='Lab' else AllDays
        for d in valid_days:
            for b in BuildingsSet:
                lh = m.addVar(vtype=GRB.BINARY)
                m.addConstr(lh <= l[(c,d)])
                m.addConstr(lh <= b_ind[(c,d,b)])
                m.addConstr(lh >= l[(c,d)]+b_ind[(c,d,b)]-1)
                off_cost = cost_between_buildings(b,OfficeBuilding,OfficeBuilding,W_intra,W_inter)
                obj_expr += off_cost * lh

m.setObjective(obj_expr, GRB.MINIMIZE)

# -------------------
# ADDING THE NEW CONSTRAINTS FOR WALKING TIME FEASIBILITY
# -------------------

for i in Instructors:
    c_i = InstructorClasses[i]
    for d in AllDays:
        # Consider each pair of classes for this instructor on the same day
        for c1, c2 in itertools.permutations(c_i, 2):
            if c1 != c2:
                # Feasible times for these classes on day d
                feasible_c1 = [(t) for (dd,t,pp) in FeasibleDayTime[c1] if dd==d]
                feasible_c2 = [(t) for (dd,t,pp) in FeasibleDayTime[c2] if dd==d]

                for t1 in feasible_c1:
                    start_c1, end_c1 = times_in_minutes[d][t1]
                    for t2 in feasible_c2:
                        start_c2, end_c2 = times_in_minutes[d][t2]
                        if start_c2 >= end_c1:
                            # gap between end_c1 and start_c2
                            gap = start_c2 - end_c1
                            # now add constraints for each room combination
                            for r1 in FeasibleRooms[c1]:
                                for r2 in FeasibleRooms[c2]:
                                    b1 = building_of_room(r1)
                                    b2 = building_of_room(r2)
                                    wtime = cost_between_buildings(b1, b2, OfficeBuilding, W_intra, W_inter)
                                    # If gap < wtime, cannot assign these pairs simultaneously
                                    if gap < wtime:
                                        m.addConstr(x[(c1,r1,d,t1)] + x[(c2,r2,d,t2)] <= 1, 
                                                    name=f"No_Infeasible_Walk_{c1}_{c2}_{d}_{t1}_{t2}_{r1}_{r2}")

m.optimize()

if m.status == GRB.OPTIMAL:
    print("Optimal solution found with objective:", m.objVal)
    
    # Extract chosen assignments
    chosen = []
    for (CRN,r,d,t) in x:
        if x[(CRN,r,d,t)].X > 0.5:
            (C,I,P,DD,dem) = ClassInfo[CRN]
            chosen.append((I, d, t, CRN, r, P))
    
    def slot_start_minutes(slot_str):
        match = re.match(r'(\d+):(\d+) (\wM)', slot_str)
        hour = int(match.group(1))
        minute = int(match.group(2))
        ampm = match.group(3)
        if ampm == 'PM' and hour < 12:
            hour += 12
        return hour*60 + minute
    
    schedule = {}
    for (I,d,t,CRN,r,P) in chosen:
        slot_str = time_slots[d][t]
        start_min = times_in_minutes[d][t][0]
        bldg = building_of_room(r)
        if I not in schedule:
            schedule[I] = {}
        if d not in schedule[I]:
            schedule[I][d] = []
        schedule[I][d].append((start_min, CRN, r, P, slot_str, bldg))
    
    # Sort each day by start time
    for I in schedule:
        for d in schedule[I]:
            schedule[I][d].sort(key=lambda x: x[0])
    
    # Print the schedule
    for I in sorted(schedule.keys()):
        print(f"\nSchedule for Instructor: {I}")
        for d in ['M','T','W','R','F']:
            if d in schedule[I]:
                print(f"  {d}:")
                for (start_min, CRN, r, P, slot_str, bldg) in schedule[I][d]:
                    print(f"    {slot_str} -> CRN {CRN} ({P}), Room {r}, Building {bldg}")
else:
    print("No optimal solution found. Status:", m.status)

